In [ ]:
import os
from utils.denoising_utils import *

FOLDER = "BSDS300/images/train/"
IMAGES = [FOLDER + img for img in os.listdir(FOLDER)][:100]
np.set_printoptions(suppress=True)
def L2(A):
    A_1D = np.reshape(A, (-1))
    return np.sqrt(A_1D.dot(A_1D) / A_1D.shape[0])
    

In [ ]:
sigmas = np.arange(0,105,5)
mus = np.arange(0,256,1)
out = np.empty((sigmas.shape[0], mus.shape[0]), dtype=np.float)

for i, sigma in enumerate(sigmas):
# for i, sigma in enumerate([50]):
    print("Sigma = {}".format(sigma))
    n = 10000000
    v = np.random.uniform(size=n)
    noise = np.random.normal(loc=np.full(n, 0), scale=sigma/255)
    unclipped = v+noise
    clipped = np.clip(unclipped, 0.0, 1.0)
#     plt.scatter(v, clipped)
    v *= 256
    v = v.astype(int, copy=False)
    clipped *= 256
    clipped = clipped.astype(int, copy=False)
    D = clipped - v
    
    for j in range(0, 256):
        out[i][j] = L2(D[v == j])
plt.plot(out[20])

In [ ]:
table = torch.from_numpy(out)
# torch.save(table, "sigma_mu_to_std.tensor")
if 'table' not in locals():
    table = torch.load("sigma_mu_to_std.tensor")
g_table = table.cuda()

dtype = torch.cuda.FloatTensor

def pred_std(image, sigma):
    image = torch.from_numpy(image).type(dtype)
#     pred_m = image.clone()
    pred = 0.0
    for channel in range(image.shape[0]):
        for x in range(image.shape[1]):
            for y in range(image.shape[2]):
                v = (g_table[int(sigma/5)][int(255.99999*image[channel][x][y])])**2
                pred += v
#                 pred_m[channel][x][y] = v
#     print(pred_m.cpu().numpy())
    return np.sqrt(pred/image.nelement())
#     return pred/image.nelement() #+ sigma*0.02

In [ ]:
stddev = []
mean = []
sigma = 50
for fname in IMAGES:
    img_pil = crop_image(get_image(fname, -1)[0], d=32)
    img_np = pil_to_np(img_pil)
    _, img_noisy_np = get_noisy_image(img_np, sigma/255)
    diff_n = img_noisy_np - img_np
    computed = L2(diff_n)*255
#     expected = diff_n.std()*255
    expected = pred_std(img_noisy_np, sigma)
    print("Computed:\t{:.2f}\tPredicted:\t{:.2f}\tDelta:  {:.2f}\tvar:\t{:.2f}\tmean:\t{:.2f}".format(
        computed, expected, computed-expected, img_noisy_np.std(), img_noisy_np.mean()
        
    ))


In [ ]:
# print(table[20][127])
sigma = 25
img_np = np.full((1,100,100), 0.0)
# img_np[0,:50,:] = 0.0
_, img_noisy_np = get_noisy_image(img_np, sigma/256)
# img_np
diff_n = img_noisy_np - img_np
print(pred_std(img_noisy_np, sigma))
# print(diff_n.std()*255)
print(L2(diff_n)*255)
print(img_np)
print(img_noisy_np)
print(diff_n)